In [40]:
import numpy as np
from mrftools import *

# uncontrolled dynamics
def phi_q(x1, x2):
    x1_x, x1_y = x1
    x2_x, x2_y = x2
    ind = ((x2_x == x1_x) and (x2_y == x1_y)) or \
            (x2_x == x1_x - 1) and (x2_y == x1_y) and (x1_x > 0) or \
            (x2_x == x1_x) and (x2_y == x1_y - 1) and (x1_y > 0) or \
            (x2_x == x1_x + 1) and (x2_y == x1_y) and (x1_x < np.sqrt(N)) or \
            (x2_x == x1_x) and (x2_y == x1_y + 1) and (x1_y < np.sqrt(N))
    return 1 if ind else 0

# from index in {0,...,N-1} to position (x,y) in {1,...,sqrt(N)}^2
def get_pos(index, N):
    index = index + 1 
    r = np.remainder(index, np.sqrt(N))
    if r == 0:
        x = np.sqrt(N)
        y = index // np.sqrt(N)
    else:
        x = r
        y = index // np.sqrt(N) + 1
    return x, y

N = 25

# build uncontrolled dynamics matrix factor
phi_q_factor = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        phi_q_factor[i, j] = phi_q(get_pos(i, N), get_pos(j, N))

mn = MarkovNet()

factor = np.array(N*[-float('inf')]) # factor = np.zeros(N)
factor[1] = 1
mn.set_unary_factor('x11', factor)

factor = np.array(N*[-float('inf')]) # factor = np.zeros(N)
factor[14] = 1
mn.set_unary_factor('x12', factor)

mn.set_unary_factor('x21', np.ones(N))
mn.set_unary_factor('x31', np.ones(N))
#mn.set_unary_factor('x41', np.ones(N))
mn.set_unary_factor('x22', np.ones(N))
mn.set_unary_factor('x32', np.ones(N))
#mn.set_unary_factor('x42', np.ones(N))

lmbda = 10
r_s = -10
r_h = -2


r_hare = np.exp(-(r_h/lmbda))
factor_h = np.ones(N)
factor_h[0] = r_hare
factor_h[4] = r_hare
factor_h[20] = r_hare
factor_h[24] = r_hare
mn.set_unary_factor('x41', factor_h)
mn.set_unary_factor('x42', factor_h)


mn.set_edge_factor(('x11', 'x21'), phi_q_factor)
mn.set_edge_factor(('x21', 'x31'), phi_q_factor)
mn.set_edge_factor(('x31', 'x41'), phi_q_factor)
mn.set_edge_factor(('x12', 'x22'), phi_q_factor)
mn.set_edge_factor(('x22', 'x32'), phi_q_factor)
mn.set_edge_factor(('x32', 'x42'), phi_q_factor)


r_stag = np.exp(-(r_s/lmbda))
factor_s = np.ones((N, N))
factor_s[(12,12)] = r_stag
mn.set_edge_factor(('x41', 'x42'), factor_s)


bp = BeliefPropagator(mn)

In [41]:
bp.infer(display='full')
bp.load_beliefs()

Iteration 0, change in messages 60.022516. Calibration disagreement: 0.147561, energy functional: 29.895011, dual obj: 29.894563
Iteration 1, change in messages 10.698740. Calibration disagreement: 0.022594, energy functional: 29.894110, dual obj: 29.894130
Iteration 2, change in messages 1.429883. Calibration disagreement: 0.000183, energy functional: 29.894071, dual obj: 29.894076
Iteration 3, change in messages 0.039336. Calibration disagreement: 0.000027, energy functional: 29.894070, dual obj: 29.894070
Iteration 4, change in messages 0.002351. Calibration disagreement: 0.000003, energy functional: 29.894069, dual obj: 29.894069
Iteration 5, change in messages 0.000367. Calibration disagreement: 0.000000, energy functional: 29.894069, dual obj: 29.894069
Iteration 6, change in messages 0.000026. Calibration disagreement: 0.000000, energy functional: 29.894069, dual obj: 29.894069
Iteration 7, change in messages 0.000000. Calibration disagreement: 0.000000, energy functional: 29.89

In [46]:
mn.unary_potentials

{'x11': array([-inf,   1., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf]),
 'x12': array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf,   1., -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf]),
 'x21': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.]),
 'x31': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.]),
 'x22': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.]),
 'x32': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.]),
 'x41': array([1.22140276, 1.        , 1.        , 1.        , 1.22140276,
        1.        , 1.        , 1.        , 1.        , 1. 

In [47]:
mn.edge_potentials

{('x11',
  'x21'): array([[1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0

In [42]:
np.exp(bp.var_beliefs['x11'])

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.])

In [43]:
np.exp(bp.pair_beliefs[('x11','x21')])

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.07962478, 0.08427162, 0.08454715, 0.0310018 , 0.02929232,
        0.0310018 , 0.08952715, 0.03297834, 0.0329352 , 0.0310018 ,
        0.03110316, 0.03297834, 0.03311136, 0.03297834, 0.03110316,
        0.0310018 , 0.0329352 , 0.03297834, 0.0329352 , 0.0310018 ,
        0.02929232, 0.0310018 , 0.03110316, 0.0310018 , 0.02929232],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.

In [52]:
with np.errstate(divide='ignore', invalid='ignore'):
    trans_mat = np.transpose(np.transpose(np.exp(bp.pair_beliefs[('x11','x21')])) / np.exp(bp.var_beliefs['x11']))
trans_mat

array([[       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan],
       [0.07962478, 0.08427162, 0.08454715, 0.0310018 , 0.02929232,
        0.0310018 , 0.08952715, 0.03297834, 0.0329352 , 0.0310018 ,
        0.03110316, 0.03297834, 0.03311136, 0.03297834, 0.03110316,
        0.0310018 , 0.0329352 , 0.03297834, 0.0329352 , 0.0310018 ,
        0.02929232, 0.0310018 , 0.03110316, 0.0310018 , 0.02929232],
       [       nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,   

In [ ]:
from staghunt import StagHuntMRF
test = StagHuntMRF()
test.build_model()
test.infer(inference_type='matrix')
test.compute_probabilities()